<a href="https://colab.research.google.com/github/iman370/FCC-ML-with-python/blob/main/Projects/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset=pd.read_csv(train_file_path,sep='\t', header=None, names=["type", "text"])
#train_dataset.head()

test_dataset=pd.read_csv(test_file_path,sep='\t', header=None, names=["type", "text"])
#test_dataset.tail()

In [ ]:
# Pre-process the data so 'ham' is represented by 0 and 'spam' is represented by 1.
train_dataset['type'] = train_dataset['type'].map({'spam': 'isSpam'})
train_dataset = pd.get_dummies(train_dataset, columns=['type'], prefix='', prefix_sep='')
#train_dataset.tail()

test_dataset['type'] = test_dataset['type'].map({'spam': 'isSpam'})
test_dataset = pd.get_dummies(test_dataset, columns=['type'], prefix='', prefix_sep='')
#test_dataset.tail()

In [ ]:
# Separate the features from what we want to predict
train_features = train_dataset.copy()
train_labels = train_features.pop('isSpam')

test_features = test_dataset.copy()
test_labels = test_features.pop('isSpam')

In [ ]:
# Let's check the database so far
#print(train_features['text'])
print(train_features)
train_features.describe()

**This is what I followed to pre-process the data: https://towardsdatascience.com/email-spam-detection-1-2-b0e06a5c0472

In [ ]:
# Integer encoding each word so that we can pass it through to the neural network
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

VOCAB_SIZE = 5000

# Tokenization
tokenizer = Tokenizer(num_words=VOCAB_SIZE, lower= 1, oov_token="<OOV>")
tokenizer.fit_on_texts(train_features['text'])
word_index = tokenizer.word_index
print(word_index)

In [ ]:
# Sequencing
train_sequences = tokenizer.texts_to_sequences(train_features['text'])
print(train_sequences)

In [ ]:
# Padding
MAXLEN = 100
train_data = keras.preprocessing.sequence.pad_sequences(train_sequences, MAXLEN)
print(train_data)

In [ ]:
# Now do the same with the testing data
test_sequences = tokenizer.texts_to_sequences(test_features['text']) # Sequencing
test_data = keras.preprocessing.sequence.pad_sequences(test_sequences, MAXLEN)
print(test_data)

In [ ]:
# Making the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

In [ ]:
# Compiling the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

In [ ]:
# Evaluation
results = model.evaluate(test_data, test_labels)
print(results)

In [ ]:
def encode_text(text): # We should encode our text with the same encoding used to make the model
  encoded = tokenizer.texts_to_sequences([text])
  #print(encoded[0])
  return keras.preprocessing.sequence.pad_sequences([encoded[0]], MAXLEN)[0]

encode_text("how are you doing today")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded_text = encode_text(pred_text)
  pred = np.zeros((1,100))
  pred[0] = encoded_text
  result = model.predict(pred) 
  #print(result[0])

  # The model should return a probability
  # Here, we should round the probability up or down
  # If rounded to 0 then it's ham, else it's spam

  if (result[0] >= 0.5):
    return (result[0], "spam")
  else:
    return (result[0], "ham")
  #return (prediction)

#pred_text = "how are you doing today?"
pred_text = "you have won £1000 cash! call to claim your prize."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()